In [14]:
import sys
sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
sys.path.append("/scratch/group/csce435-f24/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd
import math

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Read all files

In [15]:
#1_trial is a name of a folder containing the cali files, you may create a folder with a different name and replace the folder name here
tk = th.Thicket.from_caliperreader(glob("cali_files/*.cali"))

(2/2) Creating Thicket: 100%|██████████| 251/251 [00:02<00:00, 95.15it/s] 
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/thicket/ensemble.py:409: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace({numerical_fill_value: None}, inplace=True)


In [16]:
print(tk.show_metric_columns())

['Avg time/rank (exc)', 'Min time/rank', 'Avg time/rank', 'Min time/rank (exc)', 'Total time (exc)', 'Calls/rank (total)', 'Max time/rank (exc)', 'Calls/rank (avg)', 'Total time', 'Variance time/rank', 'Calls/rank (min)', 'Calls/rank (max)', 'Max time/rank']


In [17]:
print(tk.tree(metric_column="Avg time/rank"))

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2024.1.0

54.413 main
├─ 0.000 MPI_Comm_dup
├─ 0.000 MPI_Comm_rank
├─ 0.000 MPI_Comm_size
├─ 0.000 MPI_Finalize
├─ 0.000 MPI_Finalized
├─ 0.000 MPI_Init
├─ 0.000 MPI_Initialized
├─ 8.154 comm
│  ├─ 6.149 comm_large
│  │  ├─ 0.199 MPI_Alltoall
│  │  ├─ 0.053 MPI_Alltoallv
│  │  ├─ 0.004 MPI_Gather
│  │  ├─ 5.182 MPI_Gatherv
│  │  └─ 0.388 MPI_Scatterv
│  └─ 2.005 comm_small
│     ├─ 0.156 MPI_Bcast
│     ├─ 0.863 MPI_Gather
│     └─ 0.986 MPI_Gatherv
├─ 43.345 comp
│  ├─ 38.781 comp_large
│  └─ 4.565 comp_small
├─ 0.823 correctness_check
│  ├─ 0.088 MPI_Allreduce
│  ├─ 0.075 MPI_Bcast
│  ├─ 0.024 MPI_Recv
│  ├─ 0.371 MPI_Scatterv
│  └─ 0.000 MPI_Send
└─ 1.533 data_init_runtime
   └─ 0.953 MPI_Gatherv

Legend (Metric: Avg time/rank Min: 0.00 Max: 54.41 indices: {'profile': 25046506})
█ 48.97 - 54.41
█ 38.09 - 48.9

Group Performance data by `matrix_size` in the Thicket metadata table.

In [18]:
tk.metadata_column_to_perfdata("num_procs")
tk.metadata_column_to_perfdata("input_size")
tk.metadata_column_to_perfdata("input_type")

tk.dataframe = tk.dataframe.reset_index().set_index(["node", "num_procs", "input_size", "input_type"]).sort_index()


In [19]:
# tk.dataframe
tk.dataframe.head(20)

profile  \
node                                 num_procs input_size input_type                  
{'name': 'main', 'type': 'function'} 2         65536      perturbed      1149517614   
                                                          random         4224158877   
                                                          reverseSorted  1786010035   
                                                          sorted         1321230243   
                                               262144     perturbed      2970481539   
                                                          random         2087396112   
                                                          reverseSorted  3952095182   
                                                          sorted         1980362145   
                                               1048576    perturbed      2596038123   
                                                          random         4152151963   
                                                          reverseSorted  1484506347   
                                                          sorted         3586076911   
                                               4194304    perturbed       394595476   
                                                          random         1808835606   
                                                          reverseSorted  2484933290   
                                                          sorted         1001983860   
                                               16777216   perturbed      4160216407   
                                                          random         3730450291   
                                                          reverseSorted  2445449161   
                                                          sorted         2489975163   

                                                                         nid  \
node                                 num_procs input_size input_type           
{'name': 'main', 'type': 'function'} 2         65536      perturbed      1.0   
                                                          random         1.0   
                                                          reverseSorted  1.0   
                                                          sorted         1.0   
                                               262144     perturbed      1.0   
                                                          random         1.0   
                                                          reverseSorted  1.0   
                                                          sorted         1.0   
                                               1048576    perturbed      1.0   
                                                          random         1.0   
                                                          reverseSorted  1.0   
                                                          sorted         1.0   
                                               4194304    perturbed      1.0   
                                                          random         1.0   
                                                          reverseSorted  1.0   
                                                          sorted         1.0   
                                               16777216   perturbed      1.0   
                                                          random         1.0   
                                                          reverseSorted  1.0   
                                                          sorted         1.0   

                                                                          spot.channel  \
node                                 num_procs input_size input_type                     
{'name': 'main', 'type': 'function'} 2         65536      perturbed      regionprofile   
                                                          random         regionprofile   
                                        

In [20]:
tk.dataframe.xs(2**16, level='input_size') #Testing 

profile  \
node                                              num_procs input_type                  
{'name': 'main', 'type': 'function'}              2         perturbed      1149517614   
                                                            random         4224158877   
                                                            reverseSorted  1786010035   
                                                            sorted         1321230243   
                                                  4         perturbed      1227402650   
                                                            random         3986311396   
                                                            reverseSorted  1612356255   
                                                            sorted         3654201516   
                                                  8         perturbed      2957444312   
                                                            random         2974840148   
                                                            reverseSorted  3880157979   
                                                            sorted         1307354117   
                                                  16        perturbed      4179796591   
                                                            random          792074630   
                                                            reverseSorted  3925925500   
                                                            sorted         2660640971   
                                                  32        perturbed      1602884961   
                                                            random         2697194257   
                                                            reverseSorted  3604484051   
                                                            sorted         3934493794   
                                                  64        perturbed       153852366   
                                                            random         1931768397   
                                                            reverseSorted    44974794   
                                                            sorted         2803981334   
                                                  128       perturbed      4252208148   
                                                            random         2348572815   
                                                            reverseSorted  3987303630   
                                                            sorted         2585578006   
                                                  256       perturbed      3985384750   
                                                            random         3005474051   
                                                            reverseSorted  2487704945   
                                                            sorted         1871815099   
                                                  512       perturbed       316703003   
                                                            random         1708543384   
                                                            reverseSorted  2269603510   
                                                            sorted         1299559923   
{'name': 'MPI_Comm_dup', 'type': 'function'}      2         perturbed      1149517614   
                                                            random         4224158877   
                                                            reverseSorted  1786010035   
                                                            sorted         1321230243   
                                                  4         perturbed      1227402650   
                                                            random         3986311396   
                                                            reverseSorted  1612356255   
                                                            sorted       

In [21]:
# Define common variables
processes = [2, 4, 8, 16, 32, 64, 128, 256, 512]
matrix_sizes = [2**16, 2**18, 2**20, 2**22, 2**24, 2**26, 2**28]
input_types = ["sorted", "random", "reverseSorted", "perturbed"]

# Change font size for all plots
plt.rcParams.update({"font.size": 20})

# Define the function names
function_names = [
    "data_init_runtime",
    "correctness_check",
    "comm_large",
    "comm_small",
    "comp_large",
    "comp_small"
]

# First, generate the graphs for the metrics
for matrix_size in matrix_sizes:
    exponent_N = int(math.log2(matrix_size))

    for input_type in input_types:
        try:
            df_matrix_input = tk.dataframe.xs((matrix_size, input_type), level=('input_size', 'input_type'))
        except KeyError:
            continue

        for func_name in function_names:
            df_func = df_matrix_input[df_matrix_input['name'] == func_name]

            if df_func.empty:
                continue

            num_procs = df_func.index.get_level_values('num_procs').unique()

            # Ensure num_procs is sorted for plotting
            num_procs = sorted(num_procs)

            # Get the values for each metric
            min_values = df_func['Min time/rank']
            max_values = df_func['Max time/rank']
            avg_values = df_func['Avg time/rank']
            variance_values = df_func['Variance time/rank']

            # Plot the data
            plt.figure(figsize=(12, 6))
            plt.plot(num_procs, min_values.values, label='Min Time/rank', marker='o')
            plt.plot(num_procs, max_values.values, label='Max Time/rank', marker='s')
            plt.plot(num_procs, avg_values.values, label='Avg Time/rank', marker='^')
            plt.plot(num_procs, variance_values.values, label='Variance Time/rank', marker='d')

            plt.xlabel('Number of Processes')
            plt.ylabel('Time (s)')
            plt.title(f'{func_name} Times for Matrix Size 2^{exponent_N}, Input Type: {input_type}')
            plt.legend()
            plt.grid(True, which="both", ls="--", linewidth=0.5)
            plt.xscale('log', base=2)
            plt.xticks(num_procs, num_procs)
            plt.savefig(f'{func_name}_times_matrix_2^{exponent_N}_input_{input_type}.jpeg', format='jpeg')
            plt.close()

# Now, create the total time graphs separately
for func_name in function_names:
    for input_type in input_types:
        plt.figure(figsize=(12, 6))
        legend_entries = []
        for matrix_size in matrix_sizes:
            exponent_N = int(math.log2(matrix_size))

            try:
                df_matrix_input = tk.dataframe.xs((matrix_size, input_type), level=('input_size', 'input_type'))
            except KeyError:
                continue

            df_func = df_matrix_input[df_matrix_input['name'] == func_name]

            if df_func.empty:
                continue

            num_procs = df_func.index.get_level_values('num_procs').unique()
            num_procs = sorted(num_procs)
            total_time_values = df_func['Total time']

            plt.plot(num_procs, total_time_values.values, label=f'2^{exponent_N}', marker='o')
            legend_entries.append(f'2^{exponent_N}')

        if not legend_entries:
            plt.close()
            continue

        plt.xlabel('Number of Processes')
        plt.ylabel('Total Time (s)')
        plt.title(f'{func_name} Total Time for Input Type: {input_type}')
        plt.legend(title='Matrix Sizes')
        plt.grid(True, which="both", ls="--", linewidth=0.5)
        plt.xscale('log', base=2)
        plt.xticks(processes, processes)
        plt.savefig(f'{func_name}_total_time_input_{input_type}.jpeg', format='jpeg')
        plt.close()

print("Done")

Done
